In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import Normalizer
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import  accuracy_score, confusion_matrix, classification_report
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE 

In [2]:
#import the dataset.
dataset = pd.read_csv('winequalityN.csv', 
                      sep=',', na_values=["?"])

In [3]:
#print the first 5 rows to get an idea of the dataset.
dataset.head()

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,white,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,white,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,white,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,white,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,white,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [4]:
#information about the attributes and the variables of the dataset.
dataset.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   type                  6497 non-null   object 
 1   fixed acidity         6487 non-null   float64
 2   volatile acidity      6489 non-null   float64
 3   citric acid           6494 non-null   float64
 4   residual sugar        6495 non-null   float64
 5   chlorides             6495 non-null   float64
 6   free sulfur dioxide   6497 non-null   float64
 7   total sulfur dioxide  6497 non-null   float64
 8   density               6497 non-null   float64
 9   pH                    6488 non-null   float64
 10  sulphates             6493 non-null   float64
 11  alcohol               6497 non-null   float64
 12  quality               6497 non-null   int64  
dtypes: float64(11), int64(1), object(1)
memory usage: 660.0+ KB


In [5]:
dataset.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,6487.000000,6489.000000,6494.000000,6495.000000,6495.000000,6497.000000,6497.000000,6497.000000,6488.000000,6493.000000,6497.000000,6497.000000
mean,7.216579,0.339691,0.318722,5.444326,0.056042,30.525319,115.744574,0.994697,3.218395,0.531215,10.491801,5.818378
std,1.296750,0.164649,0.145265,4.758125,0.035036,17.749400,56.521855,0.002999,0.160748,0.148814,1.192712,0.873255
min,3.800000,0.080000,0.000000,0.600000,0.009000,1.000000,6.000000,0.987110,2.720000,0.220000,8.000000,3.000000
25%,6.400000,0.230000,0.250000,1.800000,0.038000,17.000000,77.000000,0.992340,3.110000,0.430000,9.500000,5.000000
50%,7.000000,0.290000,0.310000,3.000000,0.047000,29.000000,118.000000,0.994890,3.210000,0.510000,10.300000,6.000000
75%,7.700000,0.400000,0.390000,8.100000,0.065000,41.000000,156.000000,0.996990,3.320000,0.600000,11.300000,6.000000
max,15.900000,1.580000,1.660000,65.800000,0.611000,289.000000,440.000000,1.038980,4.010000,2.000000,14.900000,9.000000


In [6]:
#checking for duplicate rows
dups = dataset.duplicated()
dups.sum()

1168

In [7]:
#dropping the duplicate rows and checking the amount of rows after the drop
print( (dataset.shape[0]))
dataset = dataset.drop_duplicates()
print((dataset.shape[0]))

6497
5329


In [8]:
#convert the quality column into binary values, 1 for good (quality above or 7 ) and 0 for bad (quality bellow 7).
dataset['quality'] = dataset['quality'].apply(lambda x: 1 if x >= 7 else 0)
quality_column =dataset['quality'].value_counts()
quality_column

0    4318
1    1011
Name: quality, dtype: int64

In [9]:
#checking for missing values. we view all the rows that contain the missing values to determine their impact on the dataset.
dataset[pd.isnull(dataset).any(axis=1)]

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
17,white,NaN,0.660,0.48,1.2,0.029,29.0,75.0,0.98920,3.33,0.39,12.8,1
33,white,6.2,0.120,0.34,NaN,0.045,43.0,117.0,0.99390,3.42,0.51,9.0,0
54,white,6.8,0.200,0.59,0.9,0.147,38.0,132.0,0.99300,NaN,0.38,9.1,0
86,white,7.2,NaN,0.63,11.0,0.044,55.0,156.0,0.99740,3.09,0.44,8.7,0
98,white,9.8,0.360,0.46,10.5,NaN,4.0,83.0,0.99560,2.89,0.30,10.1,0
139,white,8.1,0.280,0.39,1.9,0.029,18.0,79.0,0.99230,NaN,0.52,11.8,0
174,white,NaN,0.270,0.31,17.7,0.051,33.0,173.0,0.99900,3.09,0.64,10.2,0
224,white,6.3,0.495,0.22,1.8,0.046,31.0,140.0,0.99290,3.39,NaN,10.4,0
249,white,NaN,0.410,0.14,10.4,0.037,18.0,119.0,0.99600,3.38,0.45,10.0,0
267,white,NaN,0.580,0.07,6.9,0.043,34.0,149.0,0.99440,3.34,0.57,9.7,0


In [10]:
#since there only a few rows i decided to drop the rows as they won't have an impact on the model. 
dataset= dataset.dropna()
dataset= dataset.drop(labels=['type'], axis=1)

In [11]:
#checking again if there are still missing values.
dataset[pd.isnull(dataset).any(axis=1)]

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality


In [12]:
# Separating features and target.
X = dataset.drop('quality', axis=1)
y = dataset['quality']

# Setting SMOTE and transforming the data.
smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X, y)

# Checking for the balance of the classes again.
print(y_smote.value_counts())
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X_smote, y_smote, test_size=0.2, random_state=42)


0    4290
1    4290
Name: quality, dtype: int64


In [13]:
#we will check which Scaler performs the best.
feature_scaler = StandardScaler()
#feature_scaler = Normalizer()
#feature_scaler=MaxAbsScaler()
#feature_scaler=MinMaxScaler()
X_train = feature_scaler.fit_transform(X_train)
X_test = feature_scaler.transform(X_test)

In [14]:
test_1=[5,10,18,20,50,100,500]
accuracy_score_test_1=[]
#conf_matrices_1 = []
class_reports_1 = []
for i in test_1:
    mlp_test_1 = MLPClassifier(hidden_layer_sizes=(i,), activation='logistic', alpha=1e-4,
                    solver='sgd', tol=1e-4, random_state=1,
                    learning_rate_init=.1, verbose=True)
    mlp_test_1.fit(X_train,y_train)
    predictions = mlp_test_1.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    accuracy_score_test_1.append((i,accuracy))
   # conf_matrices_1.append((i, confusion_matrix(y_test, predictions)))
    class_reports_1.append((i, classification_report(y_test, predictions)))
ac_1 = pd.DataFrame(accuracy_score_test_1, columns=['Number of Hidden Neurons', 'Accuracy'])
print(ac_1)
#for neurons, matrix in conf_matrices_1:
 #   print(f"Confusion Matrix for {neurons} neurons:")
  #  print(matrix)
for neurons, report in class_reports_1:
    print(f"Classification Report for {neurons} neurons:")
    print(report)

Iteration 1, loss = 0.64277740
Iteration 2, loss = 0.52387192
Iteration 3, loss = 0.48926543
Iteration 4, loss = 0.48097936
Iteration 5, loss = 0.47780397
Iteration 6, loss = 0.47625704
Iteration 7, loss = 0.47465413
Iteration 8, loss = 0.47298249
Iteration 9, loss = 0.47187555
Iteration 10, loss = 0.46954765
Iteration 11, loss = 0.46836316
Iteration 12, loss = 0.46731720
Iteration 13, loss = 0.46700955
Iteration 14, loss = 0.46550143
Iteration 15, loss = 0.46413484
Iteration 16, loss = 0.46321313
Iteration 17, loss = 0.46163459
Iteration 18, loss = 0.46136046
Iteration 19, loss = 0.45959447
Iteration 20, loss = 0.45875966
Iteration 21, loss = 0.45742984
Iteration 22, loss = 0.45676428
Iteration 23, loss = 0.45525810
Iteration 24, loss = 0.45486071
Iteration 25, loss = 0.45383431
Iteration 26, loss = 0.45275811
Iteration 27, loss = 0.45252174
Iteration 28, loss = 0.45176116
Iteration 29, loss = 0.45082573
Iteration 30, loss = 0.44980247
Iteration 31, loss = 0.44975115
Iteration 32, los

Iteration 114, loss = 0.40147116
Iteration 115, loss = 0.40054877
Iteration 116, loss = 0.40008179
Iteration 117, loss = 0.40128038
Iteration 118, loss = 0.40182221
Iteration 119, loss = 0.39994957
Iteration 120, loss = 0.40008068
Iteration 121, loss = 0.39975822
Iteration 122, loss = 0.40002512
Iteration 123, loss = 0.39922513
Iteration 124, loss = 0.39851104
Iteration 125, loss = 0.39833532
Iteration 126, loss = 0.39848211
Iteration 127, loss = 0.39988032
Iteration 128, loss = 0.39836251
Iteration 129, loss = 0.39845774
Iteration 130, loss = 0.39777561
Iteration 131, loss = 0.39752869
Iteration 132, loss = 0.39745947
Iteration 133, loss = 0.39702845
Iteration 134, loss = 0.39736509
Iteration 135, loss = 0.39767830
Iteration 136, loss = 0.39637888
Iteration 137, loss = 0.39669993
Iteration 138, loss = 0.39680463
Iteration 139, loss = 0.39754633
Iteration 140, loss = 0.39589109
Iteration 141, loss = 0.39632095
Iteration 142, loss = 0.39637376
Iteration 143, loss = 0.39624036
Iteration 

C:\Users\vange\AnacondaNew\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Iteration 1, loss = 0.60285996
Iteration 2, loss = 0.49513051
Iteration 3, loss = 0.48310718
Iteration 4, loss = 0.48092412
Iteration 5, loss = 0.47980094
Iteration 6, loss = 0.47928300
Iteration 7, loss = 0.47795884
Iteration 8, loss = 0.47714060
Iteration 9, loss = 0.47582839
Iteration 10, loss = 0.47561712
Iteration 11, loss = 0.47335088
Iteration 12, loss = 0.47193180
Iteration 13, loss = 0.47114980
Iteration 14, loss = 0.46996091
Iteration 15, loss = 0.46840895
Iteration 16, loss = 0.46646670
Iteration 17, loss = 0.46516935
Iteration 18, loss = 0.46289784
Iteration 19, loss = 0.46143788
Iteration 20, loss = 0.46039244
Iteration 21, loss = 0.45917660
Iteration 22, loss = 0.45748372
Iteration 23, loss = 0.45624521
Iteration 24, loss = 0.45399723
Iteration 25, loss = 0.45296758
Iteration 26, loss = 0.45172183
Iteration 27, loss = 0.44991947
Iteration 28, loss = 0.44830220
Iteration 29, loss = 0.44749233
Iteration 30, loss = 0.44602940
Iteration 31, loss = 0.44482914
Iteration 32, los

C:\Users\vange\AnacondaNew\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


...

Iteration 1, loss = 0.60857093
Iteration 2, loss = 0.50022536
Iteration 3, loss = 0.48586372
Iteration 4, loss = 0.48353549
Iteration 5, loss = 0.48298258
Iteration 6, loss = 0.48324564
Iteration 7, loss = 0.48297898
Iteration 8, loss = 0.48134703
Iteration 9, loss = 0.48138000
Iteration 10, loss = 0.47921194
Iteration 11, loss = 0.47868251
Iteration 12, loss = 0.47829101
Iteration 13, loss = 0.47786704
Iteration 14, loss = 0.47696249
Iteration 15, loss = 0.47457120
Iteration 16, loss = 0.47428984
Iteration 17, loss = 0.47350413
Iteration 18, loss = 0.47210758
Iteration 19, loss = 0.47206035
Iteration 20, loss = 0.46996672
Iteration 21, loss = 0.46852792
Iteration 22, loss = 0.46849658
Iteration 23, loss = 0.46668344
Iteration 24, loss = 0.46763489
Iteration 25, loss = 0.46457883
Iteration 26, loss = 0.46402032
Iteration 27, loss = 0.46202945
Iteration 28, loss = 0.46077205
Iteration 29, loss = 0.46047662
Iteration 30, loss = 0.45897362
Iteration 31, loss = 0.45736245
Iteration 32, los

C:\Users\vange\AnacondaNew\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Iteration 4, loss = 0.48659842
Iteration 5, loss = 0.48426366
Iteration 6, loss = 0.48494069
Iteration 7, loss = 0.48563956
Iteration 8, loss = 0.48349245
Iteration 9, loss = 0.48391950
Iteration 10, loss = 0.48401962
Iteration 11, loss = 0.48256879
Iteration 12, loss = 0.48353684
Iteration 13, loss = 0.48141276
Iteration 14, loss = 0.48205646
Iteration 15, loss = 0.48110082
Iteration 16, loss = 0.48159302
Iteration 17, loss = 0.47966155
Iteration 18, loss = 0.47988524
Iteration 19, loss = 0.47788341
Iteration 20, loss = 0.47704755
Iteration 21, loss = 0.47779755
Iteration 22, loss = 0.47458725
Iteration 23, loss = 0.47489556
Iteration 24, loss = 0.47253687
Iteration 25, loss = 0.47231569
Iteration 26, loss = 0.47075653
Iteration 27, loss = 0.46901946
Iteration 28, loss = 0.46928755
Iteration 29, loss = 0.46919009
Iteration 30, loss = 0.46556789
Iteration 31, loss = 0.46378477
Iteration 32, loss = 0.46333935
Iteration 33, loss = 0.46251301
Iteration 34, loss = 0.45972103
Iteration 35, 

C:\Users\vange\AnacondaNew\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Iteration 1, loss = 1.73997609
Iteration 2, loss = 0.79184482
Iteration 3, loss = 0.73416731
Iteration 4, loss = 0.68881877
Iteration 5, loss = 0.69138583
Iteration 6, loss = 0.61791298
Iteration 7, loss = 0.64437626
Iteration 8, loss = 0.66831869
Iteration 9, loss = 0.64454372
Iteration 10, loss = 0.63027661
Iteration 11, loss = 0.61785074
Iteration 12, loss = 0.60240517
Iteration 13, loss = 0.60881640
Iteration 14, loss = 0.64849220
Iteration 15, loss = 0.61825004
Iteration 16, loss = 0.60945157
Iteration 17, loss = 0.54788618
Iteration 18, loss = 0.60772501
Iteration 19, loss = 0.61393380
Iteration 20, loss = 0.55349584
Iteration 21, loss = 0.59064627
Iteration 22, loss = 0.57384355
Iteration 23, loss = 0.56151065
Iteration 24, loss = 0.56336212
Iteration 25, loss = 0.61269882
Iteration 26, loss = 0.53578881
Iteration 27, loss = 0.49856692
Iteration 28, loss = 0.54506613
Iteration 29, loss = 0.55255622
Iteration 30, loss = 0.53936322
Iteration 31, loss = 0.50531860
Iteration 32, los

In [20]:
test_2 = [(15,15),(20,20),(25,25),(30, 30), (50, 50), (100, 100),(70,60),(80,60,50)]
accuracy_score_test_2 = []
#conf_matrices_2 = []
class_reports_2 = []
for i in test_2:
    mlp_test_2 = MLPClassifier(hidden_layer_sizes=(i), activation='logistic', alpha=1e-4,
                    solver='sgd', tol=1e-4, random_state=1,
                    learning_rate_init=.2, verbose=True)
    mlp_test_2.fit(X_train,y_train)
    predictions = mlp_test_2.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    accuracy_score_test_2.append((i,accuracy))
    #conf_matrices_2.append((i, confusion_matrix(y_test, predictions)))
    class_reports_2.append((i, classification_report(y_test, predictions)))
ac_2 = pd.DataFrame(accuracy_score_test_2, columns=['Number of Hidden Layers', 'Accuracy'])
print(ac_2)
#for neurons, matrix in conf_matrices_2:
    #print(f"Confusion Matrix for {neurons} neurons:")
    #print(matrix)
for neurons, report in class_reports_2:
    print(f"Classification Report for {neurons} hidden layers and neurons:")
    print(report)

Iteration 1, loss = 0.69689726
Iteration 2, loss = 0.64363298
Iteration 3, loss = 0.50798616
Iteration 4, loss = 0.48221241
Iteration 5, loss = 0.47990962
Iteration 6, loss = 0.47687864
Iteration 7, loss = 0.47507266
Iteration 8, loss = 0.47279275
Iteration 9, loss = 0.47176134
Iteration 10, loss = 0.46902170
Iteration 11, loss = 0.46575271
Iteration 12, loss = 0.46341054
Iteration 13, loss = 0.46165456
Iteration 14, loss = 0.45997748
Iteration 15, loss = 0.45794860
Iteration 16, loss = 0.45788350
Iteration 17, loss = 0.45480071
Iteration 18, loss = 0.45225570
Iteration 19, loss = 0.45091381
Iteration 20, loss = 0.44840108
Iteration 21, loss = 0.44659194
Iteration 22, loss = 0.44235553
Iteration 23, loss = 0.44021686
Iteration 24, loss = 0.43836934
Iteration 25, loss = 0.43499839
Iteration 26, loss = 0.43323929
Iteration 27, loss = 0.43071069
Iteration 28, loss = 0.42960747
Iteration 29, loss = 0.42756395
Iteration 30, loss = 0.42547785
Iteration 31, loss = 0.42489483
Iteration 32, los

C:\Users\vange\AnacondaNew\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Iteration 2, loss = 0.57300243
Iteration 3, loss = 0.48759661
Iteration 4, loss = 0.48192451
Iteration 5, loss = 0.47699441
Iteration 6, loss = 0.47640011
Iteration 7, loss = 0.47337306
Iteration 8, loss = 0.47121585
Iteration 9, loss = 0.46945677
Iteration 10, loss = 0.46824155
Iteration 11, loss = 0.46684351
Iteration 12, loss = 0.46497218
Iteration 13, loss = 0.46221919
Iteration 14, loss = 0.46129623
Iteration 15, loss = 0.45743890
Iteration 16, loss = 0.45458559
Iteration 17, loss = 0.45384599
Iteration 18, loss = 0.45056161
Iteration 19, loss = 0.44796305
Iteration 20, loss = 0.44535622
Iteration 21, loss = 0.44361629
Iteration 22, loss = 0.44241152
Iteration 23, loss = 0.43970282
Iteration 24, loss = 0.43771869
Iteration 25, loss = 0.43513711
Iteration 26, loss = 0.43351694
Iteration 27, loss = 0.43186501
Iteration 28, loss = 0.42804415
Iteration 29, loss = 0.42512638
Iteration 30, loss = 0.42343595
Iteration 31, loss = 0.42011609
Iteration 32, loss = 0.41825712
Iteration 33, lo

C:\Users\vange\AnacondaNew\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Iteration 6, loss = 0.47657977
Iteration 7, loss = 0.47484759
Iteration 8, loss = 0.47233827
Iteration 9, loss = 0.47302351
Iteration 10, loss = 0.46972259
Iteration 11, loss = 0.46771771
Iteration 12, loss = 0.46598731
Iteration 13, loss = 0.46527711
Iteration 14, loss = 0.46259353
Iteration 15, loss = 0.46255401
Iteration 16, loss = 0.46033186
Iteration 17, loss = 0.45859892
Iteration 18, loss = 0.45714770
Iteration 19, loss = 0.45515925
Iteration 20, loss = 0.45458172
Iteration 21, loss = 0.45178040
Iteration 22, loss = 0.45066809
Iteration 23, loss = 0.44932856
Iteration 24, loss = 0.44643056
Iteration 25, loss = 0.44530028
Iteration 26, loss = 0.44290683
Iteration 27, loss = 0.43969874
Iteration 28, loss = 0.43776549
Iteration 29, loss = 0.43528686
Iteration 30, loss = 0.43219677
Iteration 31, loss = 0.42879553
Iteration 32, loss = 0.42814313
Iteration 33, loss = 0.42306449
Iteration 34, loss = 0.42143226
Iteration 35, loss = 0.41884066
Iteration 36, loss = 0.41670293
Iteration 37

...

C:\Users\vange\AnacondaNew\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Iteration 3, loss = 0.49311577
Iteration 4, loss = 0.48490551
Iteration 5, loss = 0.48194700
Iteration 6, loss = 0.48146098
Iteration 7, loss = 0.48211971
Iteration 8, loss = 0.47912728
Iteration 9, loss = 0.47869409
Iteration 10, loss = 0.47598290
Iteration 11, loss = 0.47524109
Iteration 12, loss = 0.47391245
Iteration 13, loss = 0.47324406
Iteration 14, loss = 0.47048060
Iteration 15, loss = 0.46949981
Iteration 16, loss = 0.46775345
Iteration 17, loss = 0.46849546
Iteration 18, loss = 0.46572268
Iteration 19, loss = 0.46530880
Iteration 20, loss = 0.46210538
Iteration 21, loss = 0.46009152
Iteration 22, loss = 0.45891177
Iteration 23, loss = 0.45712481
Iteration 24, loss = 0.45405733
Iteration 25, loss = 0.45419642
Iteration 26, loss = 0.45028076
Iteration 27, loss = 0.44770813
Iteration 28, loss = 0.44662659
Iteration 29, loss = 0.44446001
Iteration 30, loss = 0.44291257
Iteration 31, loss = 0.44038972
Iteration 32, loss = 0.43712115
Iteration 33, loss = 0.43600917
Iteration 34, l

C:\Users\vange\AnacondaNew\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Iteration 2, loss = 0.69712385
Iteration 3, loss = 0.69516245
Iteration 4, loss = 0.68244304
Iteration 5, loss = 0.58809184
Iteration 6, loss = 0.49833284
Iteration 7, loss = 0.48523158
Iteration 8, loss = 0.48413539
Iteration 9, loss = 0.48220034
Iteration 10, loss = 0.48256135
Iteration 11, loss = 0.48018533
Iteration 12, loss = 0.47861823
Iteration 13, loss = 0.47918365
Iteration 14, loss = 0.47757100
Iteration 15, loss = 0.47664862
Iteration 16, loss = 0.47646470
Iteration 17, loss = 0.47550035
Iteration 18, loss = 0.47541762
Iteration 19, loss = 0.47274691
Iteration 20, loss = 0.47437459
Iteration 21, loss = 0.47122221
Iteration 22, loss = 0.47255104
Iteration 23, loss = 0.47018580
Iteration 24, loss = 0.46770978
Iteration 25, loss = 0.46866150
Iteration 26, loss = 0.46663788
Iteration 27, loss = 0.46651453
Iteration 28, loss = 0.46449171
Iteration 29, loss = 0.46451628
Iteration 30, loss = 0.46274926
Iteration 31, loss = 0.46006309
Iteration 32, loss = 0.46022017
Iteration 33, lo

C:\Users\vange\AnacondaNew\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [22]:
test_3 = [0.001,0.01, 0.05, 0.2, 0.5, 0.6, 0.8]
accuracy_score_test_3 = []
class_reports_3 = []
for i in test_3:
    mlp_test_3 = MLPClassifier(hidden_layer_sizes=(70,60), activation='logistic', alpha=1e-4,
                        solver='sgd', tol=1e-4, random_state=1,
                        learning_rate_init=i, verbose=False)
    mlp_test_3.fit(X_train, y_train)
    predictions = mlp_test_3.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    accuracy_score_test_3.append((i,accuracy))
    class_reports_3.append((i, classification_report(y_test, predictions)))
ac_3 = pd.DataFrame(accuracy_score_test_3, columns=['Learning Rate', 'Accuracy'])
print(ac_3)
for act, report in class_reports_3:
    print(f"Classification Report for {act} learning rate:")
    print(report)

C:\Users\vange\AnacondaNew\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\vange\AnacondaNew\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\vange\AnacondaNew\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\vange\AnacondaNew\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\vange\AnacondaNew\lib\site-packages\sklearn\neural_netw

   Learning Rate  Accuracy
0          0.001  0.748252
1          0.010  0.769231
2          0.050  0.798368
3          0.200  0.828671
4          0.500  0.864219
5          0.600  0.853147
6          0.800  0.854895
Classification Report for 0.001 learning rate:
              precision    recall  f1-score   support

           0       0.76      0.72      0.74       856
           1       0.74      0.77      0.76       860

    accuracy                           0.75      1716
   macro avg       0.75      0.75      0.75      1716
weighted avg       0.75      0.75      0.75      1716

Classification Report for 0.01 learning rate:
              precision    recall  f1-score   support

           0       0.81      0.71      0.75       856
           1       0.74      0.83      0.78       860

    accuracy                           0.77      1716
   macro avg       0.77      0.77      0.77      1716
weighted avg       0.77      0.77      0.77      1716

Classification Report for 0.05 learni

C:\Users\vange\AnacondaNew\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [26]:
mlp = MLPClassifier(hidden_layer_sizes=(70,60), activation='relu', alpha=1e-4,
                    solver='sgd', tol=1e-4, random_state=1,
                    learning_rate_init=0.5, verbose=True)

mlp.fit(X_train, y_train)
predictions = mlp.predict(X_test)
accuracy_score(y_test, predictions)

Iteration 1, loss = 0.47842169
Iteration 2, loss = 0.43191478
Iteration 3, loss = 0.42028978
Iteration 4, loss = 0.40667085
Iteration 5, loss = 0.39863893
Iteration 6, loss = 0.38742506
Iteration 7, loss = 0.38048112
Iteration 8, loss = 0.37303096
Iteration 9, loss = 0.36212901
Iteration 10, loss = 0.36054495
Iteration 11, loss = 0.35023979
Iteration 12, loss = 0.33723047
Iteration 13, loss = 0.33335934
Iteration 14, loss = 0.32928997
Iteration 15, loss = 0.32395615
Iteration 16, loss = 0.32506729
Iteration 17, loss = 0.30505809
Iteration 18, loss = 0.31973470
Iteration 19, loss = 0.31145958
Iteration 20, loss = 0.29598548
Iteration 21, loss = 0.29085641
Iteration 22, loss = 0.29104925
Iteration 23, loss = 0.27882131
Iteration 24, loss = 0.27223375
Iteration 25, loss = 0.27556864
Iteration 26, loss = 0.26410757
Iteration 27, loss = 0.25740577
Iteration 28, loss = 0.24247990
Iteration 29, loss = 0.24554964
Iteration 30, loss = 0.24728003
Iteration 31, loss = 0.22678240
Iteration 32, los

0.844988344988345

In [27]:
Y_pred = mlp.predict(X_test)
accuracy_score(y_test, Y_pred)

0.844988344988345

In [28]:
print(classification_report(y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.89      0.79      0.84       856
           1       0.81      0.90      0.85       860

    accuracy                           0.84      1716
   macro avg       0.85      0.84      0.84      1716
weighted avg       0.85      0.84      0.84      1716



In [29]:
print(confusion_matrix(y_test, Y_pred))

[[675 181]
 [ 85 775]]
